# Student Alcohol Consumption

### Introduction:

This time you will download a dataset from the UCI.

### Step 1. Import the necessary libraries

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import requests
from pyspark.sql.types import BooleanType

In [2]:
spark = SparkSession.builder.master("local[1]").appName('alchol').getOrCreate()

22/09/13 14:14:16 WARN Utils: Your hostname, xkeyscore resolves to a loopback address: 127.0.1.1; using 192.168.1.8 instead (on interface wlp0s20f3)
22/09/13 14:14:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/13 14:14:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/Students_Alcohol_Consumption/student-mat.csv).

In [2]:
url = "https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/Students_Alcohol_Consumption/student-mat.csv"
data_content = requests.get(url)

with open("student_mat.csv","w") as f:
    f.write(data_content.text)

### Step 3. Assign it to a variable called df.

In [3]:
df = spark.read.options(header=True, inferSchema=True).csv("student_mat.csv")
df.columns

['school',
 'sex',
 'age',
 'address',
 'famsize',
 'Pstatus',
 'Medu',
 'Fedu',
 'Mjob',
 'Fjob',
 'reason',
 'guardian',
 'traveltime',
 'studytime',
 'failures',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic',
 'famrel',
 'freetime',
 'goout',
 'Dalc',
 'Walc',
 'health',
 'absences',
 'G1',
 'G2',
 'G3']

### Step 4. For the purpose of this exercise slice the dataframe from 'school' until the 'guardian' column

### Step 5. Create a lambda function that will capitalize strings.

In [38]:
capatalizer = udf(lambda x: x.capitalize())

### Step 6. Capitalize both Mjob and Fjob

In [39]:
df.select("Mjob","Fjob",initcap("Mjob").alias("Cap_Mjob"),initcap("Fjob").alias("Cap_Fjob"),capatalizer(col("school"))).show()

+--------+--------+--------+--------+----------------+
|    Mjob|    Fjob|Cap_Mjob|Cap_Fjob|<lambda>(school)|
+--------+--------+--------+--------+----------------+
| at_home| teacher| At_home| Teacher|              Gp|
| at_home|   other| At_home|   Other|              Gp|
| at_home|   other| At_home|   Other|              Gp|
|  health|services|  Health|Services|              Gp|
|   other|   other|   Other|   Other|              Gp|
|services|   other|Services|   Other|              Gp|
|   other|   other|   Other|   Other|              Gp|
|   other| teacher|   Other| Teacher|              Gp|
|services|   other|Services|   Other|              Gp|
|   other|   other|   Other|   Other|              Gp|
| teacher|  health| Teacher|  Health|              Gp|
|services|   other|Services|   Other|              Gp|
|  health|services|  Health|Services|              Gp|
| teacher|   other| Teacher|   Other|              Gp|
|   other|   other|   Other|   Other|              Gp|
|  health|

### Step 7. Print the last elements of the data set.

In [10]:
df.tail(10)

[Row(school='MS', sex='F', age=18, address='R', famsize='GT3', Pstatus='T', Medu=2, Fedu=2, Mjob='at_home', Fjob='other', reason='other', guardian='mother', traveltime=2, studytime=3, failures=0, schoolsup='no', famsup='no', paid='yes', activities='no', nursery='yes', higher='yes', internet='no', romantic='no', famrel=5, freetime=3, goout=3, Dalc=1, Walc=3, health=4, absences=2, G1=10, G2=9, G3=10),
 Row(school='MS', sex='F', age=18, address='R', famsize='GT3', Pstatus='T', Medu=4, Fedu=4, Mjob='teacher', Fjob='at_home', reason='reputation', guardian='mother', traveltime=3, studytime=1, failures=0, schoolsup='no', famsup='yes', paid='yes', activities='yes', nursery='yes', higher='yes', internet='yes', romantic='yes', famrel=4, freetime=4, goout=3, Dalc=2, Walc=2, health=5, absences=7, G1=6, G2=5, G3=6),
 Row(school='MS', sex='F', age=19, address='R', famsize='GT3', Pstatus='T', Medu=2, Fedu=3, Mjob='services', Fjob='other', reason='course', guardian='mother', traveltime=1, studytime=3,

### Step 8. Did you notice the original dataframe is still lowercase? Why is that? Fix it and capitalize Mjob and Fjob.

In [41]:
df1 = df.select('school',
 'sex',
 'age',
 'address',
 'famsize',
 'Pstatus',
 'Medu',
 'Fedu',
 'reason',
 'guardian',
 'traveltime',
 'studytime',
 'failures',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic',
 'famrel',
 'freetime',
 'goout',
 'Dalc',
 'Walc',
 'health',
 'absences',
 'G1',
 'G2',
 'G3',capatalizer(col("Mjob")), capatalizer(col("Fjob")))

### Step 9. Create a function called majority that returns a boolean value to a new column called legal_drinker (Consider majority as older than 17 years old)

In [18]:
def majority(age):
    if age >17:
        return True
    return False

In [22]:
majority_udf = udf(lambda z: majority(z),BooleanType())

In [28]:
df1 = df1.withColumn("legal_drinker",majority_udf(col("age")))

### Step 10. Multiply every number of the dataset by 10. 
##### I know this makes no sense, don't forget it is just an exercise

In [75]:
for field in df1.schema.fields:
    if str(field.dataType) in ['DoubleType()', 'FloatType()', 'LongType()', 'IntegerType()', 'DecimalType()']:
        df1 = df1.withColumn(str(field.name), col(str(field.name))*10)
        

In [76]:
df.show(10)

+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| at_home| teacher|    course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       3|    4|   1|   1|     3|       6|  5|  6|  6|
|    GP|  F| 17|    

In [77]:
df1.show(10)

+------+---+----+-------+-------+-------+----+----+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+----+----+----+--------------+--------------+
|school|sex| age|address|famsize|Pstatus|Medu|Fedu|    reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences|  G1|  G2|  G3|<lambda>(Mjob)|<lambda>(Fjob)|
+------+---+----+-------+-------+-------+----+----+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+----+----+----+--------------+--------------+
|    GP|  F|1800|      U|    GT3|      A| 400| 400|    course|  mother|       200|      200|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|   400|     300|  400| 100| 100|   300|     600| 5